In [9]:
type BoxStm = Box<Stm>;
type BoxExp = Box<Exp>;
type BoxExpList = Box<ExpList>;

#[derive(Clone)]
enum Stm {
    Compound(BoxStm, BoxStm),
    Assign { id: String, exp: BoxExp },
    Print(BoxExpList),
}

impl Stm {
    fn box_compound(s1: BoxStm, s2: BoxStm) -> BoxStm {
        Box::new(Stm::Compound(s1, s2))
    }

    fn box_assign(id: String, exp: BoxExp) -> BoxStm {
        Box::new(Stm::Assign { id, exp })
    }

    fn box_print(expList: BoxExpList) -> BoxStm {
        Box::new(Stm::Print(expList))
    }
}

#[derive(Clone)]
enum BinOp {
    Plus,
    Minus,
    Times,
    Div,
}

#[derive(Clone)]
enum Exp {
    Id(String),
    Num(i32),
    Op {
        left: BoxExp,
        op: BinOp,
        right: BoxExp,
    },
    Eseq(BoxStm, BoxExp),
}

impl Exp {
    fn box_id(id: String) -> BoxExp {
        Box::new(Exp::Id(id))
    }

    fn box_num(num: i32) -> BoxExp {
        Box::new(Exp::Num(num))
    }

    fn box_op(left: BoxExp, op: BinOp, right: BoxExp) -> BoxExp {
        Box::new(Exp::Op { left, op, right })
    }

    fn box_eseq(stm: BoxStm, exp: BoxExp) -> BoxExp {
        Box::new(Exp::Eseq(stm, exp))
    }
}

#[derive(Clone)]
enum ExpList {
    Pair { head: BoxExp, tail: BoxExpList },
    Last(BoxExp),
}

impl ExpList {
    fn box_pair(head: BoxExp, tail: BoxExpList) -> BoxExpList {
        Box::new(ExpList::Pair { head, tail })
    }
    fn box_last(exp: BoxExp) -> BoxExpList {
        Box::new(ExpList::Last(exp))
    }
}

// a := 5 + 3 ; b := ( print ( a , a - 1) , 10 * a) ; print ( b )
let prog = Stm::box_compound(
    Stm::box_assign(
        "a".to_owned(),
        Exp::box_op(Exp::box_num(5), BinOp::Plus, Exp::box_num(3)),
    ),
    Stm::box_compound(
        Stm::box_assign(
            "b".to_owned(),
            Exp::box_eseq(
                Stm::box_print(ExpList::box_pair(
                    Exp::box_id("a".to_owned()),
                    ExpList::box_last(Exp::box_op(
                        Exp::box_id("a".to_owned()),
                        BinOp::Minus,
                        Exp::box_num(1),
                    )),
                )),
                Exp::box_op(
                    Exp::box_num(10),
                    BinOp::Times,
                    Exp::box_id("a".to_owned()),
                ),
            ),
        ),
        Stm::box_print(ExpList::box_last(Exp::box_id("b".to_owned()))),
    ),
);


（1） 写一个函数 int maxargs(A_stm)，告知给定语句中任意子表达式内的 print 语句的参数个数。例如，maxargs(prog) 的值是 2。

In [11]:
fn maxargs(stm: BoxStm) -> usize {
    match *stm {
        Stm::Compound(stm1, stm2) => maxargs(stm1).max(maxargs(stm2)),
        Stm::Assign { exp, .. } => max_exp_args(exp),
        Stm::Print(expList) => count_exp_list(expList),
    }
}

fn max_exp_args(exp: BoxExp) -> usize {
    match *exp {
        Exp::Op { left, right, .. } => max_exp_args(left).max(max_exp_args(right)),
        Exp::Eseq(stm, exp) => maxargs(stm).max(max_exp_args(exp)),
        _ => 0,
    }
}

fn count_exp_list(expList: BoxExpList) -> usize {
    match *expList {
        ExpList::Pair { head, tail } => max_exp_args(head).max(1 + count_exp_list(tail)),
        ExpList::Last(exp) => 1 + max_exp_args(exp),
    }
}


println!("print maxargs is {:?}.", maxargs(prog.clone()));


print maxargs is 2.


（2） 写一个函数 void interp(A_stm), 对一个用这种直线式程序语言写的程序进行“解释”，为了用“函数式程序设计”风格来编写该函数（这种风格不使用赋值语句），要在声明局部变量的同时对它进行初始化。

In [12]:
type OptBoxTable = Option<Box<Table>>;

struct Table {
    id: String,
    value: i32,
    tail: OptBoxTable,
}

impl Table {
    fn lookup(t: OptBoxTable, id: &str) -> (Option<i32>, OptBoxTable) {
        if let Some(t) = t {
            if t.id == id {
                return (Some(t.value), Some(t));
            }
            if let Some(tail) = t.tail {
                return Self::lookup(Some(tail), id);
            }
            (None, Some(t))
        } else {
            (None, t)
        }
    }

    fn update(tail: OptBoxTable, id: String, value: i32) -> OptBoxTable {
        Some(Box::new(Self { id, value, tail }))
    }
}

type IntAndTable = (i32, OptBoxTable);

fn inter_exp(exp: BoxExp, t: OptBoxTable) -> IntAndTable {
    match *exp {
        Exp::Id(id) => {
            let (v, t) = Table::lookup(t, id.as_ref());
            (
                v.unwrap_or_else(|| panic!("Error: {} is not defined.", id)),
                t,
            )
        }
        Exp::Num(v) => (v, t),
        Exp::Op { left, op, right } => {
            let (lv, t) = inter_exp(left, t);
            let (rv, t) = inter_exp(right, t);
            let v = match op {
                BinOp::Plus => lv + rv,
                BinOp::Minus => lv - rv,
                BinOp::Times => lv * rv,
                BinOp::Div => lv / rv,
            };
            (v, t)
        }
        Exp::Eseq(stm, exp) => inter_exp(exp, interp_stm(stm, t)),
    }
}

fn print_exp_list(expList: BoxExpList, t: OptBoxTable) -> OptBoxTable {
    match *expList {
        ExpList::Pair { head, tail } => {
            let (v, t) = inter_exp(head, t);
            print!("{} ", v);
            print_exp_list(tail, t)
        }
        ExpList::Last(exp) => {
            let (v, t) = inter_exp(exp, t);
            println!("{}", v);
            t
        }
    }
}

fn interp_stm(stm: BoxStm, t: OptBoxTable) -> OptBoxTable {
    match *stm {
        Stm::Compound(stm1, stm2) => interp_stm(stm2, interp_stm(stm1, t)),
        Stm::Assign { id, exp } => {
            let (v, t) = inter_exp(exp, t);
            Table::update(t, id, v)
        }
        Stm::Print(expList) => print_exp_list(expList, t),
    }
}


interp_stm(prog, None);

8 7
80
